In [1]:
import tensorflow as tf
from tensorflow_tts.inference import TFAutoModel, AutoConfig

2024-07-15 01:50:16.817006: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-15 01:50:16.839517: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-15 01:50:17.202512: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/s44504/miniconda3/envs/tftts2/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly vers

In [2]:
def convert_text2mel_tflite(
    model_path: str, save_name: str, config_path: str = None, use_auth_token: bool = False
) -> float:
    # load pretrained model
    config = AutoConfig.from_pretrained(config_path if config_path else model_path)
    kwargs = {"use_auth_token": use_auth_token} if use_auth_token else {}
    model = TFAutoModel.from_pretrained(
        model_path, config=config, enable_tflite_convertible=True, **kwargs
    )

    # setup model concrete function
    concrete_function = model.inference_tflite.get_concrete_function()
    converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_function])

    # specify optimizations
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.target_spec.supported_ops = [
        tf.lite.OpsSet.TFLITE_BUILTINS,  # quantize
        tf.lite.OpsSet.SELECT_TF_OPS,
    ]

    # convert and save model to TensorFlowLite
    tflite_model = converter.convert()
    with open(save_name, "wb") as f:
        f.write(tflite_model)

    size = len(tflite_model) / 1024 / 1024.0
    return size

In [3]:
def convert_vocoder_tflite(
    model_path: str, save_name: str, config_path: str = None, use_auth_token: bool = False
) -> float:
    config = AutoConfig.from_pretrained(config_path if config_path else model_path)
    kwargs = {"use_auth_token": use_auth_token} if use_auth_token else {}
    # load pretrained model
    model = TFAutoModel.from_pretrained(model_path, config=config, **kwargs)

    # setup model concrete function
    concrete_function = model.inference_tflite.get_concrete_function()
    converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_function])

    # specify optimizations
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.target_spec.supported_ops = [tf.lite.OpsSet.SELECT_TF_OPS]
    converter.target_spec.supported_types = [tf.float16]  # fp16 ops

    # convert and save model to TensorFlowLite
    tflite_model = converter.convert()
    with open(save_name, "wb") as f:
        f.write(tflite_model)

    size = len(tflite_model) / 1024 / 1024.0
    return size

In [5]:
text2mel = convert_text2mel_tflite(
    model_path="/home/s44504/lightspeech-sw-tz-victoria-ft-vocab-exp/model.h5",
    config_path="/home/s44504/lightspeech-sw-tz-victoria-ft-vocab-exp/config.yml",
    save_name="lightspeech_quant.tflite",
    use_auth_token=False,
)

vocoder = convert_vocoder_tflite(
    model_path="/home/s44504/mb-melgan-hifi-sw-tz-victoria-ft-vocab-exp/model.h5",
    config_path="/home/s44504/mb-melgan-hifi-sw-tz-victoria-ft-vocab-exp/config.yml",
    save_name="mbmelgan.tflite",
    use_auth_token=False,
)

2024-07-15 01:51:37.447940: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-15 01:51:37.448086: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-15 01:51:37.448133: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 2
2024-07-15 01:51:37.448187: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2024-07-15 01:51:37.448514: I tensorflow/compiler/xla/stream_executor/

In [6]:
print(f"Text2mel: {text2mel} MBs\nVocoder: {vocoder} MBs")

Text2mel: 4.500709533691406 MBs
Vocoder: 5.052890777587891 MBs


In [7]:
from tensorflow_tts.inference import AutoProcessor

processor = AutoProcessor.from_pretrained("bookbot/lightspeech-mfa-sw-v2", use_auth_token=True)
processor.mode = "eval" # change processor from train to eval mode

/home/s44504/miniconda3/envs/tftts2/lib/python3.8/site-packages/huggingface_hub/file_download.py:671: FutureWarning: 'cached_download' is the legacy way to download files from the HF hub, please consider upgrading to 'hf_hub_download'
  warnings.warn(


In [8]:
from typing import List, Tuple

def tokenize(text: str, processor: AutoProcessor) -> List[int]:
    return processor.text_to_sequence(text)

In [9]:
def prepare_input(
    input_ids: List[str], speaker: int
) -> Tuple[tf.Tensor, tf.Tensor, tf.Tensor, tf.Tensor, tf.Tensor]:
    input_ids = tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0)
    return (
        input_ids,
        tf.convert_to_tensor([speaker], tf.int32),
        tf.convert_to_tensor([1.0], dtype=tf.float32),
        tf.convert_to_tensor([1.0], dtype=tf.float32),
        tf.convert_to_tensor([1.0], dtype=tf.float32),
    )

In [10]:
def ls_infer(
    input_ids: List[str], speaker: int, lightspeech_path: str
) -> Tuple[tf.Tensor, tf.Tensor, tf.Tensor]:
    # load model to Interpreter
    lightspeech = tf.lite.Interpreter(model_path=lightspeech_path)
    input_details = lightspeech.get_input_details()
    output_details = lightspeech.get_output_details()

    # print(input_details)

    # resize input tensors according to actual shape
    lightspeech.resize_tensor_input(input_details[0]["index"], [1, len(input_ids)])
    lightspeech.resize_tensor_input(input_details[1]["index"], [1])
    lightspeech.resize_tensor_input(input_details[2]["index"], [1])
    lightspeech.resize_tensor_input(input_details[3]["index"], [1])
    lightspeech.resize_tensor_input(input_details[4]["index"], [1])

    # allocate tensors
    lightspeech.allocate_tensors()

    input_data = prepare_input(input_ids, speaker)

    # set input tensors
    for i, detail in enumerate(input_details):
        lightspeech.set_tensor(detail["index"], input_data[i])

    # invoke interpreter
    lightspeech.invoke()

    # print(output_details)

    # return outputs
    return (
        lightspeech.get_tensor(output_details[0]["index"]),
        lightspeech.get_tensor(output_details[1]["index"]),
    )

In [11]:
def melgan_infer(melspectrogram: tf.Tensor, mb_melgan_path: str) -> tf.Tensor:
    # load model to Interpreter
    mb_melgan = tf.lite.Interpreter(model_path=mb_melgan_path)
    input_details = mb_melgan.get_input_details()
    output_details = mb_melgan.get_output_details()

    # resize input tensors according to actual shape
    mb_melgan.resize_tensor_input(
        input_details[0]["index"],
        [1, melspectrogram.shape[1], melspectrogram.shape[2]],
        strict=True,
    )

    # allocate tensors
    mb_melgan.allocate_tensors()

    # set input tensors
    mb_melgan.set_tensor(input_details[0]["index"], melspectrogram)

    # invoke interpreter
    mb_melgan.invoke()

    # return output
    return mb_melgan.get_tensor(output_details[0]["index"])

In [23]:
# text = "The quick brown fox jumps over the lazy dog, while the phoneme sounds of pheasants, quails and crickets chirp in the background."
text = "Hapo mwanzo Mungu aliumba mbingu na dunia."
# text = "Giza lilikuwa juu ya uso wa vilindi vya maji, naye Roho wa Mungu alikuwa ametulia juu ya maji."
# text = "Nimepata kupungukiwa, pia nimepata kuwa na wingi wa vitu."
input_ids = tokenize(text, processor)

In [24]:
# _, mel_output_tflite = ls_infer(
#     input_ids, speaker=0, lightspeech_path="fastspeech2_quant.tflite"
# )

mel_output_tflite, _ = ls_infer(
    input_ids, speaker=0, lightspeech_path="lightspeech_quant.tflite"
)

audio_tflite = melgan_infer(mel_output_tflite[:, :, :], mb_melgan_path="mbmelgan.tflite")[
    0, :, 0
]

2024-07-15 01:54:32.853009: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-15 01:54:32.853145: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-15 01:54:32.853211: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [25]:
mel_output_tflite.shape

(1, 238, 80)

In [26]:
audio_tflite.shape

(121856,)

In [27]:
import soundfile as sf

sf.write("./gen2.wav", audio_tflite, 44100, "PCM_16")